In this notebook we will try to answer at the following question:
# Is it possible to build a classifier to identify legendary pokémon?

In [87]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

pokemon=pd.read_csv("../dataset/pokemon.csv")

We will proceed by building a <b> decision tree </b> considering the sum of the values of the stat, the capture rate, the primary type.<br>
We will train the classifier with all the pokémon from the first to the sixth generation.<br>
We will ask the classifier to classify the seventh generation pokémon. <br> 
For the proper functioning of the training, we must add to the subset a pokémon that has a base stat of 570 like Tapu Koko.

In [88]:
subset=pd.concat([pokemon[pokemon['generation']<=6], pokemon[pokemon['name']=='Tapu Koko']])

In [89]:
features = ['base_total', 'capture_rate', 'type1']

In [90]:
from sklearn.preprocessing import LabelEncoder

sum_encoder = LabelEncoder()
sum_encoder.fit(subset['base_total'])

rate_encoder = LabelEncoder()
rate_encoder.fit(subset['capture_rate'])

type1_encoder = LabelEncoder()
type1_encoder.fit(subset['type1'])

subset['base_total'] = sum_encoder.transform(subset['base_total'])
subset['capture_rate'] = rate_encoder.transform(subset['capture_rate'])
subset['type1'] = type1_encoder.transform(subset['type1'])

In [91]:
from sklearn import tree

X = subset[features]

In [92]:
Y = subset['is_legendary']

In [93]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)

In [94]:
def filter(obj):
    transformed_obj = obj[['base_total','capture_rate','type1']]
    transformed_obj['base_total'] = sum_encoder.transform([transformed_obj['base_total']])[0]
    transformed_obj['capture_rate'] = rate_encoder.transform([transformed_obj['capture_rate']])[0]
    transformed_obj['type1'] = type1_encoder.transform([transformed_obj['type1']])[0]
    return transformed_obj

In [97]:
seventh_gen=pokemon[pokemon['generation']>6]
seventh_gen.set_index('name',inplace=True)
arr=clf.predict([filter(seventh_gen.loc['Necrozma'])]) #try to change the pokémon name!
if arr[0]:
    print('legendary')
else:
    print('no legendary')

legendary
